In [1]:
import pandas as pd

df = pd.read_csv('/data/ephemeral/home/ay_yolo/pseudo_label_7360_025.csv')
df.head()

,Unnamed: 0,PredictionString,image_id
0,0,0 0.28938689827919006 481.24188232421875 610.2...,test/0000.jpg
1,1,0 0.26969292759895325 641.159912109375 648.989...,test/0001.jpg
2,2,0 0.5905788540840149 881.4190673828125 466.465...,test/0002.jpg
3,3,9 0.9233554005622864 144.21945190429688 262.50...,test/0003.jpg
4,4,0 0.5441678166389465 425.85174560546875 408.90...,test/0004.jpg


In [2]:
# NaN 값 확인
empty_rows_nan = df[df['PredictionString'].isnull()]
empty_bbox_image = list(empty_rows_nan.index)
# 결과 확인
print(empty_bbox_image)


[]


In [3]:
df.dropna(subset=['PredictionString'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
# PredictionString의 각 요소를 문자열로 변환하고, NaN은 빈 문자열로 대체한 후 띄어쓰기 기준으로 나누어 리스트로 변환
predict = df['PredictionString'].apply(lambda x: str(x).split())

# 결과 확인
print(predict)

0       [0, 0.28938689827919006, 481.24188232421875, 6...
1       [0, 0.26969292759895325, 641.159912109375, 648...
2       [0, 0.5905788540840149, 881.4190673828125, 466...
3       [9, 0.9233554005622864, 144.21945190429688, 26...
4       [0, 0.5441678166389465, 425.85174560546875, 40...
                              ...                        
4866    [0, 0.877597987651825, 293.90313720703125, 347...
4867    [0, 0.30287718772888184, 629.2179565429688, 66...
4868    [7, 0.9177708029747009, 116.20901489257812, 44...
4869    [0, 0.8905738592147827, 0.483978271484375, 239...
4870    [4, 0.9380990266799927, 374.4445495605469, 204...
Name: PredictionString, Length: 4871, dtype: object


In [5]:
# NaN 값 확인
empty_rows_nan = df[df['PredictionString'].isnull()]

# 결과 확인
print(empty_rows_nan)


Empty DataFrame
Columns: [Unnamed: 0, PredictionString, image_id]
Index: []


In [6]:
print(len(df)) #4853 -> 0.5일 때 Len

4871


In [7]:
import json

# JSON 파일 경로 설정
output_json_path = '/data/ephemeral/home/ay_yolo/yolo_dataset_ay/test.json'  # 결과를 저장할 파일

# test.json 읽기
with open(output_json_path, 'r') as f:
    test_data = json.load(f)

re_id = 4883
#bbox가 없는 test이미지 삭제
test_data['images'] = [image for image in test_data['images'] if image['id'] not in empty_bbox_image]
for i in range(len(test_data['images'])):
    test_data['images'][i]['id'] = re_id
    re_id += 1
    # print(test_data['images'][i]['id'])

In [8]:
new_test_data = []
start_id = 23144
for i in range(len(df)):
    predicts = predict[i]
    image_id = test_data['images'][i]['id'] #df['image_id'][i][5:9]
    id = int(image_id)
    for j in range(0,len(predicts),6):
        category_id = predicts[j]
        x1 = float(predicts[j+2])
        y1 = float(predicts[j+3])
        x2 = float(predicts[j+4])
        y2 = float(predicts[j+5])
        w = x2 - x1
        h = y2 - y1
        area = w*h
    
        new_item = {
        "image_id": id,
        "category_id" : int(category_id),
        "area" : area,
        "bbox" : [
            x1,
            y1,
            w,
            h
        ],
        "iscrowd" : 0,
        "id" : start_id
        }
        start_id += 1
        new_test_data.append(new_item)

In [9]:
new_test_data

[{'image_id': 4883,
  'category_id': 0,
  'area': 1364.3140873908997,
  'bbox': [481.24188232421875,
   610.2066650390625,
   27.9344482421875,
   48.83984375],
  'iscrowd': 0,
  'id': 23144},
 {'image_id': 4883,
  'category_id': 0,
  'area': 4439.010751821101,
  'bbox': [480.6799621582031,
   598.149658203125,
   72.81350708007812,
   60.964111328125],
  'iscrowd': 0,
  'id': 23145},
 {'image_id': 4883,
  'category_id': 1,
  'area': 45521.053092492744,
  'bbox': [564.6475219726562,
   105.3935317993164,
   178.657470703125,
   254.79512786865234],
  'iscrowd': 0,
  'id': 23146},
 {'image_id': 4883,
  'category_id': 1,
  'area': 11169.169860713184,
  'bbox': [464.8357849121094,
   552.6132202148438,
   105.05087280273438,
   106.321533203125],
  'iscrowd': 0,
  'id': 23147},
 {'image_id': 4883,
  'category_id': 1,
  'area': 1364.3140873908997,
  'bbox': [481.24188232421875,
   610.2066650390625,
   27.9344482421875,
   48.83984375],
  'iscrowd': 0,
  'id': 23148},
 {'image_id': 4883,
 

In [10]:
# import json

# JSON 파일 경로 설정
train_json_path = '/data/ephemeral/home/dataset/train.json'


# train_split.json 읽기
with open(train_json_path, 'r') as f:
    train_data = json.load(f)

train_data['images'] += test_data['images']
train_data['annotations'] += new_test_data
    
# 수정된 내용을 JSON 파일로 저장
with open('dino7360_pseudo_label_0.25_real.json', 'w') as f:
    json.dump(train_data, f, indent=4)
print("pseudo_labeling을 위한 json파일 수정 끝~~~~")



pseudo_labeling을 위한 json파일 수정 끝~~~~


In [12]:
import os
import shutil

# pseudo_images 폴더 생성
pseudo_dir = '/data/ephemeral/home/ay_yolo/yolo_dataset_ay/train'
os.makedirs(pseudo_dir, exist_ok=True)

train_split_dir = '/data/ephemeral/home/ay_yolo/yolo_dataset_ay/train_split/images'

# train_split_dir의 이미지 파일들을 복사
for image_info in os.listdir(train_split_dir):
    # 파일의 ID 추출 (확장자 제외)
    src_img = os.path.join(train_split_dir, image_info)
    dst_path =  os.path.join(pseudo_dir,image_info)
    shutil.copy(src_img, dst_path)

print("train_split 이미지 복사가 완료되었습니다.")

train_split 이미지 복사가 완료되었습니다.


In [3]:
import json
import os
import shutil

# test_dir 경로
test_dir = '/data/ephemeral/home/ay_yolo/yolo_dataset_ay/test'
t_dir = '/data/ephemeral/home/ay_yolo/yolo_dataset_ay'
pseudo_test_json_path = '/data/ephemeral/home/ay_yolo/yolo_dataset_ay/dino7360_pseudo_label_0.25_real.json'
pseudo_dir = '/data/ephemeral/home/ay_yolo/yolo_dataset_ay/train'


# train_split.json 읽기
with open(pseudo_test_json_path, 'r') as f:
    pseudo_test_data = json.load(f)

for i in range(len(pseudo_test_data["images"])):
    file_name = pseudo_test_data['images'][i]["file_name"]
    file_dir = file_name[0:4]
    
    if file_dir == "test":
        
        src_path = os.path.join(t_dir, file_name)
        new_id = pseudo_test_data['images'][i]["id"]
        new_id = str(new_id).zfill(4)
        # print(new_id)
        new_image_info = new_id+'.jpg'
        dst_path = os.path.join(pseudo_dir, new_image_info)
        shutil.copy(src_path, dst_path)

print("pseudo_labeling 이미지 복사가 완료되었습니다.")


pseudo_labeling 이미지 복사가 완료되었습니다.
